<a href="https://colab.research.google.com/github/hemankundu/indian-currency-recognizer/blob/master/currency_plain_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
import os
#from shutil import copy
import shutil
from random import shuffle
import tensorflow as tf
import tensorflow.keras.preprocessing as prep
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import callbacks

In [0]:
#setup drive path and data directory
drive_path = '/content/drive/My Drive/colab/currency/'
data_dir = drive_path + "dataset_plain/pending/"
classes = os.listdir(data_dir)

FileNotFoundError: ignored

In [0]:
classes

In [0]:
#creating training,  testing sb directories for each class
if not os.path.exists(drive_path + 'dataset_plain/tmp/'):
  os.mkdir(drive_path + 'dataset_plain/tmp/')
if not os.path.exists(drive_path + 'dataset_plain/tmp/currency'):
  os.mkdir(drive_path + 'dataset_plain/tmp/currency')
training_dir = 'dataset_plain/tmp/currency/training/'
testing_dir = 'dataset_plain/tmp/currency/testing/'
dev_dir = 'dataset_plain/tmp/currency/dev/'
dirs = [training_dir, dev_dir, testing_dir]

for dir_ in dirs:
  dir_ = drive_path + dir_
  if not os.path.exists(dir_):
    os.mkdir(dir_)
  for cls in classes:
    cls_dir = dir_ + cls
    if not os.path.exists(cls_dir):
      print("creating " , cls_dir)
      os.mkdir(cls_dir)


In [0]:
#set training and testing directory path
training_dir = drive_path + 'dataset_plain/tmp/currency/training/'
testing_dir = drive_path + 'dataset_plain/tmp/currency/testing/'
dev_dir = drive_path + 'dataset_plain/tmp/currency/dev/'
dirs = [training_dir, dev_dir, testing_dir]

In [0]:
#split data into traing and testing
def split_data(data_dir, training_dir, dev_dir, testing_dir, split):
  dirs = [training_dir, dev_dir, testing_dir]
  if sum(split) > 1.0:
    print('Invalid split')
    return
  for cls in classes:
    print(cls)
    data = os.listdir(data_dir + cls)
    shuffle(data)
    i = 0
    for dir_, sp in zip(dirs, split):
      for _ in range(int(len(data)*sp)):
        shutil.copy(data_dir + cls + '/' + data[i], dir_+ cls)
        if (len(data)/(i+1))%5 == 0:
          print((len(data))-(len(data)/(i+1)))
        i += 1


In [0]:
split_data(data_dir, training_dir, dev_dir, testing_dir, [.9, .0, 0.1])

In [0]:
#hepler function for cloning virtual instance's tmp folder to drive's tmp folder
import os, glob, shutil

def make_dir(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def copy_dir(source_item, destination_item):
    if os.path.isdir(source_item):
        make_dir(destination_item)
        sub_items = glob.glob(source_item + '/*')
        for sub_item in sub_items:
            copy_dir(sub_item, destination_item + '/' + sub_item.split('/')[-1])
    else:
      try:
        shutil.copy(source_item, destination_item)
      except Exception:
        pass

copy_dir("/tmp", drive_path+'tmp')

In [0]:
#print count of files present in each directory
for dir_ in dirs:
  #print(dir_, len(os.listdir(dir_)))
  for cls in classes:
    print(dir_ + cls, len(os.listdir(dir_ + cls)))

In [0]:
#crop the images, so that we get only currency part of it, w/o any background
import cv2
from os.path import basename
from glob import glob

def get_contours(img):
    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret, thresh = cv2.threshold(imgray, 150, 255, 0)
    img2, contours, hierarchy = cv2.findContours(thresh, 1, 2)

    # filter contours that are too large or small
    size = get_size(img)
    contours = [cc for cc in contours if contourOK(cc, size)]
    return contours

def get_size(img):
    ih, iw = img.shape[:2]
    return iw * ih

def contourOK(cc, size=1000000):
    x, y, w, h = cv2.boundingRect(cc)
    if w < 50 or h < 50: return False # too narrow or wide is bad
    area = cv2.contourArea(cc)
    rt = area < (size * 0.8) and area > 200
    if not rt:
      print("ignoring..")
    return rt

def find_boundaries(img, contours):
    ih, iw = img.shape[:2]
    minx = iw
    miny = ih
    maxx = 0
    maxy = 0

    for cc in contours:
        x, y, w, h = cv2.boundingRect(cc)
        if x < minx: minx = x
        if y < miny: miny = y
        if x + w > maxx: maxx = x + w
        if y + h > maxy: maxy = y + h

    return (minx, miny, maxx, maxy)

def crop(img, boundaries):
    minx, miny, maxx, maxy = boundaries
    return img[miny:maxy, minx:maxx]

def process_image(path, fname):
    img = cv2.imread(path + fname)
    contours = get_contours(img)
    #cv2.drawContours(img, contours, -1, (0,255,0)) # draws contours, good for debugging
    bounds = find_boundaries(img, contours)
    cropped = crop(img, bounds)
    #if get_size(cropped) < 400: return # too small
    cv2.imwrite(path + fname, cropped)

In [0]:
#calling crop function
for cls in classes:
  print(cls)
  path = drive_path + 'dataset_plain/tmp/currency/training/' + cls
  l = os.listdir(path)
  for f in l:
    try:
      process_image( path + '/' , f)
    except Exception:
      pass

In [0]:
#helper function to delete data from tmp folder when training is done
for cls in classes:
  print(cls)
  path = drive_path + 'dataset_plain/tmp/currency/training/' + cls
  l = os.listdir(path)
  for f in l:
    try:
      if os.path.getsize(path + '/' + f) == 0:
        os.remove(path + '/' + f)
        print("Deleted..")
    except Exception:
      pass

In [0]:
dim = 128

In [0]:
#setup data generators for image dataa augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(horizontal_flip=True , shear_range = 0.2, fill_mode='nearest' , rescale=1.0/255.0, 
                                      rotation_range=10, width_shift_range= 0.1, height_shift_range=0.1, zoom_range=0.1)
train_generator = train_datagen.flow_from_directory(training_dir, target_size=(dim, dim), batch_size=64)
test_datagen = ImageDataGenerator(horizontal_flip=True , shear_range = 0.2, fill_mode='nearest' , rescale=1.0/255.0, 
                                      rotation_range=10, width_shift_range= 0.1, height_shift_range=0.1, zoom_range=0.1)
test_generator = test_datagen.flow_from_directory(testing_dir, target_size=(dim, dim), batch_size=64)
#test_datagen = ImageDataGenerator(rescale=1.0/255.0, rotation_range=40, width_shift_range= 0.1, height_shift_range=0.1, zoom_range=0.1 )
#test_generator = test_datagen.flow_from_directory(testing_dir, target_size=(dim, dim), batch_size=64)



Found 1159 images belonging to 9 classes.
Found 126 images belonging to 9 classes.


In [0]:
#defining the model
from tensorflow.keras import regularizers
l2 = 0.01
built_model = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(dim, dim, 3)),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.MaxPool2D(), 
tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.MaxPool2D(),  
tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
tf.keras.layers.Dropout(rate=0.2),
tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.MaxPool2D(), 
tf.keras.layers.Dropout(rate=0.2),
tf.keras.layers.Conv2D(128, 3,  activation='relu'),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.MaxPool2D(), 
tf.keras.layers.Dropout(rate=0.2), 
tf.keras.layers.Conv2D(256, 3,  activation='relu'),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.MaxPool2D(), 
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(1024, activation='relu'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(512, activation='relu'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(256, activation='relu'),
tf.keras.layers.Dense(9, activation='softmax'),          
])


In [0]:
built_model.summary()

In [0]:
model_name = "cnn_l2_5"

In [0]:
#to save only the model architecture
with open(drive_path + "dataset_plain/" + model_name + ".json", "w") as f:
  f.write(built_model.to_json())

In [0]:
#to load a model from saved json file
json_file = open(drive_path + "dataset_plain/" + model_name + ".json", 'r')
loaded_model_json = json_file.read()
json_file.close()
built_model = tf.keras.models.model_from_json(loaded_model_json)

In [0]:
#to load a model weights, with two modes: full is manually saved version, and auto is auto-saved version
saved_version = "full"
loaded_model = tf.keras.models.load_model(drive_path + "dataset_plain/"  + model_name + '_'+ saved_version +'.h5')
for layer_loaded, layer_built in zip(loaded_model.layers, built_model.layers):
   layer_built.set_weights(layer_loaded.get_weights())
#loaded_model.summary()

In [0]:
#compile model
built_model.compile(optimizer=Adam(learning_rate=0.0001) , loss='categorical_crossentropy', metrics=['acc'])

In [0]:
#setup callback to auto-save model weights after each epoch
checkpoint = callbacks.ModelCheckpoint(drive_path + "dataset_plain/"  + model_name +'_auto.h5', monitor='loss', verbose=1, save_best_only=False, mode='min', overwrite=True)
callbacks_list = [checkpoint]

In [0]:
#fit model, can be run using loop or specifying epoch
history = []
for i in range(1):
  print("\nloop: ", i)
  history.append(built_model.fit_generator(train_generator,
                              epochs=100,
                              verbose=1,
                              validation_data=test_generator,
                              callbacks=callbacks_list))

In [0]:
#to manually save the model
built_model.save(drive_path + "dataset_plain/" + model_name + '_full.h5')
built_model.save_weights(drive_path + "dataset_plain/" + model_name + '_w.h5')

In [0]:
#convert the model to tflite version
from tensorflow import lite
converter = lite.TFLiteConverter.from_keras_model_file(drive_path + "dataset_plain/" + model_name + '_full.h5')
tfmodel = converter.convert()
open (drive_path + "dataset_plain/" + model_name + "_full.tflite" , "wb").write(tfmodel)

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 40 variables.
INFO:tensorflow:Converted 40 variables to const ops.


8693060

In [0]:
#!pip install --upgrade tensorflow

In [0]:
#evaluation
built_model.evaluate_generator(test_generator, verbose=1)

2/2 [==============================] - 1s 470ms/step - loss: 0.2998 - acc: 0.9365


[0.29984238743782043, 0.93650794]

In [0]:
#in case while model fitting epochs is used instead of loops
history = history[0]

In [0]:
# PLOT LOSS AND ACCURACY
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt


acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")


plt.title('Training and validation loss')

In [0]:
#actual indecies of classes that the datagenerator is using
#here we inerchange key with value of the dictionary
classes_form_gen = dict([(v, k) for (k, v) in test_generator.class_indices.items()])

In [0]:
#To upload own image and test the prediction
import numpy as np
from google.colab import files
from keras.preprocessing import image
import cv2
uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(dim, dim))
  # img = cv2.imread(path)
  # img = cv2.resize(img, (dim, dim))
  x = image.img_to_array(img)
  x = x/255.0
  x = np.expand_dims(x, axis=0)
  #print(x)
  #images = np.vstack([x])
  prediction = built_model.predict(x)
  print(prediction)
  prediction = prediction[0]
  bestclass = 0
  for i in range(len(classes_form_gen)):
    if (prediction[i] > prediction[bestclass]):
      bestclass = i
  print('\nI think this is a ' + classes_form_gen[bestclass] + ' with ' + str(prediction[bestclass] * 100) + '% confidence.')
  #print("I think this is a ", classes_form_gen[prediction.index(max(prediction))])
  

Saving cap0_1.jpg to cap0_1.jpg
[[4.6499739e-08 2.4789435e-04 2.0394781e-07 4.2128550e-09 5.5666597e-08
  7.3576297e-07 1.5899574e-04 9.9959046e-01 1.6079370e-06]]

I think this is a twohundred with 99.95904564857483% confidence.


In [0]:
#to check number of missclassified data of each class
import numpy as np
from google.colab import files
from keras.preprocessing import image
for cls in classes_form_gen.values():
  cnt = 0
  print(cls, "-"*(30-len(str(cls))))
  path = drive_path + 'dataset_plain/tmp/currency/testing/' + cls
  l = os.listdir(path)
  for f in l:
    img = image.load_img(path + '/' + f, target_size=(dim, dim))
    # img = cv2.imread(path + '/' + f)
    # img = cv2.resize(img, (dim, dim))
    x = image.img_to_array(img)
    x = x/255.0
    x = np.expand_dims(x, axis=0)
    prediction = built_model.predict(x)
    prediction = prediction[0]
    bestclass = 0
    for i in range(len(classes_form_gen)):
      if (prediction[i] > prediction[bestclass]):
        bestclass = i
    #print(classes_form_gen[bestclass], prediction[bestclass])
    if classes_form_gen[bestclass] == cls:
      cnt += 1
    else:
      print(classes_form_gen[bestclass], int(prediction[train_generator.class_indices[cls]]*100))
  print("Correct Incorrect: ", cnt, len(l)-cnt)
    

fifty_2 -----------------------
Correct Incorrect:  15 0
five --------------------------
Correct Incorrect:  14 0
fivehundred_2 -----------------
Correct Incorrect:  13 0
hundred -----------------------
Correct Incorrect:  3 0
hundred_2 ---------------------
hundred 2
hundred 20
Correct Incorrect:  14 2
ten_2 -------------------------
Correct Incorrect:  26 0
twenty_2 ----------------------
Correct Incorrect:  15 0
twohundred --------------------
fivehundred_2 2
fivehundred_2 4
fivehundred_2 1
Correct Incorrect:  12 3
twothousand -------------------
Correct Incorrect:  9 0


In [0]:
print(test_generator.class_indices[0])

In [0]:
classes

{0: 'fifty_2',
 1: 'five',
 2: 'fivehundred_2',
 3: 'hundred',
 4: 'hundred_2',
 5: 'ten_2',
 6: 'twenty_2',
 7: 'twohundred',
 8: 'twothousand'}

In [0]:
!nvidia-smi       #Get Details about GPU

Wed Oct 30 04:43:39 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!cat /proc/cpuinfo      #Get CPU Information

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 4
core id		: 0
cpu cores	: 2
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4600.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	: